In [ ]:
!pip install keras keras_preprocessing

import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from keras.utils import to_categorical
from keras_preprocessing.image import load_img, ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
import kagglehub

# Dataset Download
path = kagglehub.dataset_download("jonathanoheix/face-expression-recognition-dataset")
TRAIN_DIR = os.path.join(path, "images", "train")
TEST_DIR = os.path.join(path, "images", "validation")

# Dataset to DataFrame
def createdataframe(dir):
    image_paths, labels = [], []
    for label in os.listdir(dir):
        for image in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, image))
            labels.append(label)
    return image_paths, labels

train = pd.DataFrame()
train['image_paths'], train['labels'] = createdataframe(TRAIN_DIR)

test = pd.DataFrame()
test['image_paths'], test['labels'] = createdataframe(TEST_DIR)

# Image Preprocessing
def extract_features(image_paths):
    features = []
    for image_path in tqdm(image_paths):
        img = load_img(image_path, grayscale=True, target_size=(48, 48))
        img = np.array(img)
        features.append(img)
    features = np.array(features).reshape(len(features), 48, 48, 1)
    return features

x_train = extract_features(train['image_paths']) / 255.0
x_test = extract_features(test['image_paths']) / 255.0

# Encode Labels and One-hot
le = LabelEncoder()
y_train = to_categorical(le.fit_transform(train['labels']))
y_test = to_categorical(le.transform(test['labels']))

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)

# Callbacks
early = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

# Model Architecture
model = Sequential()

model.add(Conv2D(128, (3,3), activation='relu', input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.30))

model.add(Conv2D(512, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.30))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.40))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.30))
model.add(Dense(7, activation='softmax'))

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=100,
    validation_data=(x_test, y_test),
    callbacks=[early, reduce_lr, checkpoint]
)


  0%|          | 0/28821 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


  0%|          | 0/7066 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100


...

451/451 ━━━━━━━━━━━━━━━━━━━━ 17s 38ms/step - accuracy: 0.6198 - loss: 1.0171 - val_accuracy: 0.6386 - val_loss: 0.9674 - learning_rate: 2.0000e-04
Epoch 32/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 17s 38ms/step - accuracy: 0.6178 - loss: 1.0122 - val_accuracy: 0.6076 - val_loss: 1.0348 - learning_rate: 2.0000e-04
Epoch 33/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 19s 41ms/step - accuracy: 0.6212 - loss: 1.0113 - val_accuracy: 0.6337 - val_loss: 0.9712 - learning_rate: 2.0000e-04
Epoch 34/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - accuracy: 0.6203 - loss: 1.0179 - val_accuracy: 0.6370 - val_loss: 0.9791 - learning_rate: 2.0000e-04
Epoch 35/100
450/451 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.6263 - loss: 1.0025

451/451 ━━━━━━━━━━━━━━━━━━━━ 17s 38ms/step - accuracy: 0.6263 - loss: 1.0025 - val_accuracy: 0.6459 - val_loss: 0.9447 - learning_rate: 4.0000e-05
Epoch 36/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 19s 41ms/step - accuracy: 0.6252 - loss: 0.9923 - val_accuracy: 0.6439 - val_loss: 0.9546 - learning_rate: 4.0000e-05
Epoch 37/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 17s 39ms/step - accuracy: 0.6274 - loss: 0.9965 - val_accuracy: 0.6434 - val_loss: 0.9507 - learning_rate: 4.0000e-05
Epoch 38/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 17s 37ms/step - accuracy: 0.6227 - loss: 0.9992 - val_accuracy: 0.6451 - val_loss: 0.9429 - learning_rate: 4.0000e-05
Epoch 39/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 18s 41ms/step - accuracy: 0.6268 - loss: 0.9915 - val_accuracy: 0.6411 - val_loss: 0.9581 - learning_rate: 4.0000e-05
Epoch 40/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 17s 38ms/step - accuracy: 0.6285 - loss: 0.9997 - val_accuracy: 0.6445 - val_loss: 0.9447 - learning_rate: 4.0000e-05
Epoch 41/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 17s 38ms/st

451/451 ━━━━━━━━━━━━━━━━━━━━ 17s 39ms/step - accuracy: 0.6265 - loss: 0.9868 - val_accuracy: 0.6465 - val_loss: 0.9425 - learning_rate: 4.0000e-05
Epoch 44/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.6299 - loss: 0.9824 - val_accuracy: 0.6404 - val_loss: 0.9546 - learning_rate: 4.0000e-05
Epoch 45/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.6324 - loss: 0.9745

451/451 ━━━━━━━━━━━━━━━━━━━━ 17s 37ms/step - accuracy: 0.6324 - loss: 0.9745 - val_accuracy: 0.6469 - val_loss: 0.9373 - learning_rate: 8.0000e-06
Epoch 46/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 17s 39ms/step - accuracy: 0.6239 - loss: 0.9885 - val_accuracy: 0.6459 - val_loss: 0.9380 - learning_rate: 8.0000e-06
Epoch 47/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.6358 - loss: 0.9755

451/451 ━━━━━━━━━━━━━━━━━━━━ 17s 37ms/step - accuracy: 0.6358 - loss: 0.9756 - val_accuracy: 0.6472 - val_loss: 0.9414 - learning_rate: 8.0000e-06
Epoch 48/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 17s 37ms/step - accuracy: 0.6373 - loss: 0.9783 - val_accuracy: 0.6469 - val_loss: 0.9418 - learning_rate: 8.0000e-06
Epoch 49/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 17s 37ms/step - accuracy: 0.6297 - loss: 0.9722 - val_accuracy: 0.6461 - val_loss: 0.9368 - learning_rate: 1.6000e-06
Epoch 50/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 20s 37ms/step - accuracy: 0.6304 - loss: 0.9802 - val_accuracy: 0.6472 - val_loss: 0.9378 - learning_rate: 1.6000e-06
Epoch 51/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 17s 37ms/step - accuracy: 0.6232 - loss: 0.9879 - val_accuracy: 0.6463 - val_loss: 0.9387 - learning_rate: 1.6000e-06
Epoch 52/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 17s 38ms/step - accuracy: 0.6359 - loss: 0.9653 - val_accuracy: 0.6468 - val_loss: 0.9383 - learning_rate: 1.6000e-06
Epoch 53/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 20s 37ms/st

In [ ]:
from google.colab import drive
from keras.models import load_model

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Save model in both formats (choose one or both)

# Option A: Save as HDF5 (.h5)
model.save('/content/drive/MyDrive/emotion_model.h5')

# Option B: Save in TensorFlow SavedModel format
model.save('/content/drive/MyDrive/emotion_model.keras')

# (Optional) To load later:
# model = load_model('/content/drive/MyDrive/emotion_model.h5')
# OR
# model = load_model('/content/drive/MyDrive/emotion_model.keras')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
